In [17]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [46]:
from __future__ import print_function, division
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import os
import time
import copy
import warnings
import numpy as np
import pandas
import tensorflow as tf
import matplotlib.pyplot as plt
plt.ion()
warnings.filterwarnings("ignore")

In [47]:
data_dir ='/content/drive/MyDrive/ML_Project/fruits-360/fruits-360/Test'
test_transform = transforms.Compose([transforms.Resize(200),transforms.CenterCrop(224),transforms.ToTensor()])
test_data = datasets.ImageFolder(data_dir, transform=test_transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=16,shuffle=False,num_workers=2)



In [49]:
data_dir ='/content/drive/MyDrive/ML_Project/fruits-360/fruits-360/Training'
train_transform = transforms.Compose([transforms.Resize(200) , transforms.CenterCrop(224), transforms.ToTensor()])
train_data = datasets.ImageFolder(data_dir,transform=train_transform)
trainloader = torch.utils.data.DataLoader(train_data,batch_size=16,shuffle=False,num_workers=2)

In [50]:
labels = train_data.classes

In [51]:
batch_size = 500   # Number of samples in each batch
epoch_num = 4      # Number of epochs to train the network
lr = 0.0005        # Learning rate


In [52]:
#VGG16 Architecture Build 



class VGG_architecture(nn.Module):
  def __init__(self,in_channels=3,num_classes=1000):
    super(VGG_architecture,self).__init__()
    self.layer1 = vgg_conv_block([in_channels,64], [64,64], [3,3], [1,1], 2, 2)
    self.layer2 = vgg_conv_block([64,128], [128,128], [3,3], [1,1], 2, 2)
    self.layer3 = vgg_conv_block([128,256,256], [256,256,256], [3,3,3], [1,1,1], 2, 2)
    self.layer4 = vgg_conv_block([256,512,512], [512,512,512], [3,3,3], [1,1,1], 2, 2)
    self.layer5 = vgg_conv_block([512,512,512], [512,512,512], [3,3,3], [1,1,1], 2, 2)

    self.layer1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
    self.layer1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

    self.layer2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
    self.layer2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
    
    self.layer3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
    self.layer3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
    self.layer3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

    self.layer4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
    self.layer4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.layer4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

    self.layer5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.layer5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    self.layer5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
    # max pooling (kernel_size, stride)
    self.pool = nn.MaxPool2d(2, 2)
    # fully conected layers:
    self.fully_connected6 = nn.Linear(7*7*512, 4096)
    self.fully_connected7 = nn.Linear(4096, 4096)
    self.fully_connected8 = nn.Linear(4096, 1000)

    def forward(self, x, training=True):
        x = F.relu(self.layer1_1(x))
        x = F.relu(self.layer1_2(x))
        x = self.pool(x)
        x = F.relu(self.layer2_1(x))
        x = F.relu(self.layer2_2(x))
        x = self.pool(x)
        x = F.relu(self.layer3_1(x))
        x = F.relu(self.layer3_2(x))
        x = F.relu(self.layer3_3(x))
        x = self.pool(x)
        x = F.relu(self.layer4_1(x))
        x = F.relu(self.layer4_2(x))
        x = F.relu(self.layer4_3(x))
        x = self.pool(x)
        x = F.relu(self.layer5_1(x))
        x = F.relu(self.layer5_2(x))
        x = F.relu(self.layer5_3(x))
        x = self.pool(x)
        x = x.view(-1, 7 * 7 * 512)
        x = F.relu(self.fully_connected6(x))
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.fully_connected7(x))
        x = F.dropout(x, 0.5, training=training)
        x = self.fully_connected8(x)
        return x


    # Fully Connected layers
    self.layer6 = vgg_fc_layer(7*7*512, 4096)
    self.layer7 = vgg_fc_layer(4096, 4096)

    # Final layer
    self.layer8 = nn.Linear(4096, num_classes)

In [55]:
def predict(self, x):
        # a function to predict the labels of a batch of inputs
        x = F.softmax(self.forward(x, training=False))
        return x    

In [56]:
def accuracy(self, x, y):
        prediction = self.predict(x)
        maxs, indices = torch.max(prediction, 1)
        acc = 100 * torch.sum(torch.eq(indices.float(), y.float()).float())/y.size()[0]
        return acc.cpu().data[0]


In [57]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VGG_architecture(in_channels=3, num_classes=len(labels)).to(device)
rate = nn.CrossEntropyLoss()
learn_speed = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learn_speed)
total_steps = len(trainloader)
number_of_epochs = 1

In [ ]:
for epoch in range(number_of_epochs):
    for i, (images, category) in enumerate(trainloader):
      __,outputs = model(images)
      loss = rate(outputs, category )
      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch+1, number_of_epochs, i+1, total_steps, loss.item()))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

Epoch [1/1], Step [1/390], Loss: 2.7814
Epoch [1/1], Step [2/390], Loss: 0.0000
Epoch [1/1], Step [3/390], Loss: 0.0000
Epoch [1/1], Step [4/390], Loss: 0.0000
Epoch [1/1], Step [5/390], Loss: 0.0000
Epoch [1/1], Step [6/390], Loss: 0.0000
Epoch [1/1], Step [7/390], Loss: 0.0000
Epoch [1/1], Step [8/390], Loss: 0.0000
